# Lab: Programmatically Grant App Roles to an Application (Script)

In this lab, you will learn how to programmatically grant app roles to an application using Microsoft Graph PowerShell SDK. This is a common task when setting up service-to-service authentication scenarios.

## Objectives
- Find the necessary identifiers for app role assignment
- Grant an app role to an application using PowerShell
- Verify the assignment in the Entra portal
- Extend your existing script to work with applications

## Prerequisites
- Microsoft Graph PowerShell SDK installed
- An application registered in Entra ID
- Permissions to manage application permissions

Make sure you have completed the previous labs and have the necessary authentication set up.

## Step 0: Specify your user name

You are all working in the same tenant. Edit the following code cell to specify your user name (for example `userone`, `usertwo`...).

Don't forget to run it. That user name will be used for naming resources during the labs.

In [ ]:
# Specify your user name (for example "userone", "usertwo"...)
$your_username = '<USERNAME>'

## Task 1: Finding the Required IDs

To grant an app role, you need to identify:
1. The **service principal ID** of your application ("PSTraining Script (YOUR_USERNAME)")
2. The **service principal ID** of Microsoft Graph
3. The **ID of the app role** you want to assign ("Application.Read.All")

Let's start by finding these values:

In [ ]:
# Connect to Microsoft Graph if not already connected
# Uncomment the following line if needed
Connect-MgGraph -Scopes "Application.ReadWrite.All", "Directory.ReadWrite.All", "AppRoleAssignment.ReadWrite.All"
(Get-MgContext).Scopes

# Find the service principal for our application
$appName = "PSTraining Script ({0})" -f $your_username
$sp = Get-MgServicePrincipal -Filter "displayName eq '$appName'"
$sp | Select-Object Id, DisplayName, AppId

# Find the Microsoft Graph service principal
$graph = Get-MgServicePrincipal -Filter "displayName eq 'Microsoft Graph'"
$graph | Select-Object Id, DisplayName, AppId

# Find the Application.Read.All app role
$appRole = $graph.AppRoles | Where-Object { $_.Value -eq "Application.Read.All" }
$appRole | Select-Object Id, Value, DisplayName

## Task 2: Granting the App Role

Now that we have the necessary IDs, we can grant the app role to our application using the `New-MgServicePrincipalAppRoleAssignment` cmdlet.

In [ ]:
# Create parameters for app role assignment
$params = @{
  'PrincipalId' = $sp.Id       # ID of our application's service principal
  'ResourceId'  = $graph.Id    # ID of Microsoft Graph service principal
  'AppRoleId'   = $appRole.Id  # ID of the Application.Read.All app role
}

# Grant the app role
New-MgServicePrincipalAppRoleAssignment -ServicePrincipalId $graph.Id -BodyParameter $params | 
  Format-List Id, AppRoleId, CreatedDateTime, PrincipalDisplayName, PrincipalId, PrincipalType, ResourceDisplayName

## Task 3: Verify the Assignment

After granting the app role, you should verify it was assigned correctly. You can do this using the Azure or Entra portal, or through PowerShell.

### Through PowerShell:

In [ ]:
# Check app role assignments for our service principal
Get-MgServicePrincipalAppRoleAssignment -ServicePrincipalId $sp.Id | 
    Select-Object AppRoleId, PrincipalId, ResourceDisplayName, Id

### Through Portal:

1. Navigate to the [Entra portal](https://entra.microsoft.com)
2. Go to **App registrations**
3. Find and select your application ("PSTraining Script (YOUR_USERNAME)")
4. Click on **API permissions**
5. Verify that "Application.Read.All" appears in the list of granted permissions

## Task 4: Extend Previous Script to List Applications

In previous lab, you created a script to list users and groups. Now, modify that script to also list applications. 

Test the script.

## Bonus Task: Grant Multiple App Roles at Once

If you want to grant multiple app roles to an application at once, you can use a loop to assign each role. Try granting both "Application.Read.All" and "Directory.Read.All" to your application:

In [ ]:
# Define the app roles you want to assign
$requiredAppRoles = @("Application.Read.All", "Directory.Read.All")

# Loop through each role and assign it
foreach ($roleName in $requiredAppRoles) {
    $role = $graph.AppRoles | Where-Object { $_.Value -eq $roleName }
    
    if ($role) {
        $params = @{
            'PrincipalId' = $sp.Id
            'ResourceId'  = $graph.Id
            'AppRoleId'   = $role.Id
        }
        
        Write-Host "Granting $roleName app role..." -ForegroundColor Cyan
        try {
            New-MgServicePrincipalAppRoleAssignment -ServicePrincipalId $sp.Id -BodyParameter $params -ErrorAction Stop
            
            Write-Host "Successfully granted $roleName" -ForegroundColor Green
        }
        catch {
            # The role might already be assigned
            Write-Host "Error granting $roleName - it may already be assigned" -ForegroundColor Yellow
            Write-Host $_.Exception.Message
        }
    }
    else {
        Write-Host "Could not find app role with value $roleName" -ForegroundColor Red
    }
}

## Summary

In this lab, you learned how to:
- Find necessary service principal and app role IDs
- Grant app roles to applications programmatically
- Verify app role assignments
- Extend a script to work with applications

These skills are essential when setting up automation for application permissions in Entra ID, especially in CI/CD pipelines or when managing large numbers of applications.